In [3]:
import streamlit as st
import os
from langchain.tools import DuckDuckGoSearchRun
from langchain import hub
from langchain.agents import create_openai_functions_agent
from langchain_openai import AzureChatOpenAI
from langgraph.prebuilt import create_react_agent
from langchain_core.pydantic_v1 import BaseModel
from langchain.chains.openai_functions import create_structured_output_runnable
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import Annotated, Any, Dict, Optional, Sequence, TypedDict, List, Tuple
import operator

In [4]:
import dotenv
dotenv.load_dotenv(override=True)

True

In [17]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="deepseek-coder",
    api_key=os.environ.get("OPENAI_API_KEY"),
    base_url=os.environ.get("OPENAI_API_BASE"),
)

In [57]:
from langchain.chains.openai_functions import create_structured_output_runnable
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers.base import BaseOutputParser

class Code(BaseModel):
    code: str = Field(
        description="Detailed optimized error-free code on the provided requirements"
    )

class CodeSnippetParser(BaseOutputParser):
    def parse(self, output: str) -> str:
        languages = ["python", "java"]
        for language in languages:
            if f"```{language}" in output:
                output = output[
                    output.find(f"```{language}") + len(f"```{language}") :
                ]
                output = output[: output.find("```")]
                break
        else:
            output = "No code block found"
            print("Error: No code block found")
        return output

    @property
    def _type(self) -> str:
        return "code_snippet_parser"
    

code_gen_prompt = ChatPromptTemplate.from_template(
    '''**Role**: You are a expert software {language} programmer. You need to develop {language} code
**Task**: As a programmer, you are required to complete the function. Use a Chain-of-Thought approach to break
down the problem, create pseudocode, and then write the code in {language} language. Ensure that your code is
efficient, readable, and well-commented.

**Instructions**:
1. **Understand and Clarify**: Make sure you understand the task.
2. **Algorithm/Method Selection**: Decide on the most efficient way.
3. **Pseudocode Creation**: Write down the steps you will follow in pseudocode.
4. **Code Generation**: Translate your pseudocode into executable {language} code

*REQURIEMENT*
{requirement}'''
)

parser = CodeSnippetParser()
coder = create_structured_output_runnable(
    Code, llm, code_gen_prompt, output_parser=parser
)


In [58]:
code_ = coder.invoke({"requirement":"Check if a number is prime number", "language": "python"})

In [59]:
print(code_)


import math

def is_prime(number):
    """
    Check if a number is a prime number.
    
    Parameters:
    number (int): The number to check.
    
    Returns:
    bool: True if the number is prime, False otherwise.
    """
    # Numbers less than 2 are not prime
    if number < 2:
        return False
    
    # Check divisibility from 2 to the square root of the number
    for i in range(2, int(math.sqrt(number)) + 1):
        if number % i == 0:
            return False
    
    # If no divisors were found, the number is prime
    return True

# Example usage:
print(is_prime(29))  # Should return True
print(is_prime(15))  # Should return False



In [65]:
class Test(BaseModel):
    
    Input: List[List] = Field(
        description="Input for Test cases to evaluate the provided code"
    )
    Output: List[List] = Field(
        description="Expected Output for Test cases to evaluate the provided code"
    )



from langchain.chains.openai_functions import create_structured_output_runnable
from langchain_core.prompts import ChatPromptTemplate

test_gen_prompt = ChatPromptTemplate.from_template(
    '''**Role**: As a tester, your task is to create Basic and Simple test cases based on provided Requirement and {language} Code. 
These test cases should encompass Basic, Edge scenarios to ensure the code's robustness, reliability, and scalability.
**1. Basic Test Cases**:
- **Objective**: Basic and Small scale test cases to validate basic functioning 
**2. Edge Test Cases**:
- **Objective**: To evaluate the function's behavior under extreme or unusual conditions.
**Instructions**:
- Implement a comprehensive set of test cases based on requirements.
- Pay special attention to edge cases as they often reveal hidden bugs.
- Only Generate Basics and Edge cases which are small
- Avoid generating Large scale and Medium scale test case. Focus only small, basic test-cases
**REQURIEMENT**
{requirement}
**Code**
{code}
**Output Format**
```plaintext
Input=[[input1], [input2], ...]
Output=[[output1], [output2], ...]
```

# For example:
**REQURIEMENT**
Generate fibbinaco series

**Code**
def generate_fibonacci(n):
    """
    Generate the first n numbers in the Fibonacci series.

    Parameters:
    n (int): The number of Fibonacci numbers to generate.

    Returns:
    list: A list containing the first n Fibonacci numbers.
    """
    # Initialize the first two numbers of the series
    a, b = 0, 1
    # Initialize an empty list to store the Fibonacci series
    fib_series = []
    
    # Loop to generate the Fibonacci series
    for _ in range(n):
        # Append the current number to the series
        fib_series.append(a)
        # Calculate the next number in the series
        a, b = b, a + b
    
    # Return the generated Fibonacci series
    return fib_series

# Example usage:
# Generate the first 10 Fibonacci numbers
print(generate_fibonacci(10))

**Output**
Input=[[0], [1], [2], [5], [10], [-1], [1.5], ['three'], [None]]
Output=[[[]], [[0]], [[0, 1]], [[0, 1, 1, 2, 3]], [[0, 1, 1, 2, 3, 5, 8, 13, 21, 34]], [[]], [[]], [[]], [[]]]
'''
)
import ast
class TestCaseParser(BaseOutputParser):
    # Input: List[List] = Field(
    #     description="Input for Test cases to evaluate the provided code"
    # )
    # Output: List[List] = Field(
    #     description="Expected Output for Test cases to evaluate the provided code"
    # )

    def parse(self, output: str) -> Tuple[List[List], List[List]]:
        if f"```plaintext" in output:
            output = output[
                output.find(f"```plaintext") + len(f"```plaintext") :
            ]
            output = output[: output.find("```")]
        try:
            lines = output.split('\n')
            for line in lines:
                if line.startswith("Input="):
                    input_string = line[6:]
                    print(input_string)
                    Input = ast.literal_eval(input_string)
                if line.startswith("Output="):
                    output_string = line[7:]
                    print(output_string)
                    Output = ast.literal_eval(output_string)
            return (Input, Output)
        except Exception as e:
            print(e)
            return (None, None)

    @property
    def _type(self) -> str:
        return "code_snippet_parser"
parser = TestCaseParser()
tester_agent = create_structured_output_runnable(
    Test, llm, test_gen_prompt, output_parser=parser
)


In [66]:
test_ = tester_agent.invoke({"requirement":"Generate fibbinaco series","code": code_, "language": "python"})
print(test_)


Input=[[2], [3], [4], [5], [10], [11], [12], [13], [17], [19]]
Output=[[True], [True], [False], [True], [False], [True], [False], [True], [True], [True]]

Expecting value: line 1 column 3 (char 2)
(None, None)
